In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import os
import re

In [2]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [3]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [5]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [6]:
json_election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_election_file.append(json.loads(line))

In [7]:
dict_tweets={}
i=0
for tweet in json_election_file:
    dict_tweets[i]=tweet
    i+=1
tweets = pd.DataFrame.from_dict(dict_tweets, orient='index')
tweets.reset_index(level=0, inplace=False)
tweets.head(10)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,entities,favorited,retweeted,filter_level,lang,timestamp_ms,possibly_sensitive,display_text_range,extended_tweet,extended_entities
0,Mon Nov 30 12:19:06 +0000 2020,1333385061723324419,1333385061723324419,"RT @briebriejoy: Are ""hunger and nutrition"" no...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746468,NaN,NaN,NaN,NaN
1,Mon Nov 30 12:19:06 +0000 2020,1333385061719298049,1333385061719298049,"It is fair, and urgent. Esp. in KY, SC and Maine.","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746467,NaN,NaN,NaN,NaN
2,Mon Nov 30 12:19:06 +0000 2020,1333385061970837504,1333385061970837504,RT @Rasmussen_Poll: Almost half (47%) of U.S. ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746527,NaN,NaN,NaN,NaN
3,Mon Nov 30 12:19:06 +0000 2020,1333385062155513857,1333385062155513857,RT @Goss30Goss: When Trump is convicted &amp; ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746571,NaN,NaN,NaN,NaN
4,Mon Nov 30 12:19:06 +0000 2020,1333385062193057799,1333385062193057799,"literally ""fall""\nhttps://t.co/uij0ZiPde0","<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1606738746580,False,NaN,NaN,NaN
5,Mon Nov 30 12:19:06 +0000 2020,1333385062176468993,1333385062176468993,@_Mercian @realDonaldTrump Framed another way....,"<a href=""http://twitter.com/download/android"" ...",True,1.332471e+18,1332471058360918017,1.272608e+18,1272607681757446145,...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1606738746576,NaN,"[27, 140]",{'full_text': '@_Mercian @realDonaldTrump Fram...,NaN
6,Mon Nov 30 12:19:06 +0000 2020,1333385062029651969,1333385062029651969,RT @MollieKatzen: If you're not already suppor...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746541,NaN,NaN,NaN,NaN
7,Mon Nov 30 12:19:06 +0000 2020,1333385062440726531,1333385062440726531,Today’s observations:\nBiden did not break his...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,en,1606738746639,NaN,NaN,{'full_text': 'Today’s observations: Biden did...,NaN
8,Mon Nov 30 12:19:06 +0000 2020,1333385062583324687,1333385062583324687,"RT @ACJJustice: So, we wake up to see that #St...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,"{'hashtags': [{'text': 'StopTheStupid', 'indic...",False,False,low,en,1606738746673,NaN,NaN,NaN,NaN
9,Mon Nov 30 12:19:06 +0000 2020,1333385062679764997,1333385062679764997,RT @Lrihendry: I will never give up on Trump.,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,None,...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,en,1606738746696,NaN,NaN,NaN,NaN


In [8]:
pd.set_option('display.max_columns', None)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2505 entries, 0 to 2504
Data columns (total 36 columns):
created_at                   2505 non-null object
id                           2505 non-null int64
id_str                       2505 non-null object
text                         2505 non-null object
source                       2505 non-null object
truncated                    2505 non-null bool
in_reply_to_status_id        370 non-null float64
in_reply_to_status_id_str    370 non-null object
in_reply_to_user_id          391 non-null float64
in_reply_to_user_id_str      391 non-null object
in_reply_to_screen_name      391 non-null object
user                         2505 non-null object
geo                          4 non-null object
coordinates                  4 non-null object
place                        24 non-null object
contributors                 0 non-null object
retweeted_status             1724 non-null object
quoted_status_id             520 non-null float64
quoted_sta

In [9]:
#Normalize functions:

#NO SALEN HASHTAGS NI DEL TEXT NI DE ENTITIES.MIRAR RETWEETED STATUS Y TIENE QUE DEVOLVER
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        try:
            hashtags_info = data['entities']['hashtags']
        except:
            hashtags_info = data['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'display_text_range', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'quote_count','lang', 
                    'possibly_sensitive', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data

#retweeted_status only exists if the tweet has been retweeted by someone. If not, extended_tweet exists if Truncated is true.
#If both are not the case, we use directly the information given in the tweet.
def get_tweet_data(data: pd.DataFrame) -> pd.DataFrame:
    #  Tweet| Username | Date | Hashtags | Likes | Retweets | Url
    tweets_text =[]
    tweets_username =[]
    tweets_date = []
    tweets_hashtags = []
    tweets_likes = []
    tweets_retweets = []
    tweets_url = []
    for tweet in range(len(data)):
        
        if str(data.iloc[tweet]['retweeted_status'])=='nan':
            if(data["truncated"][tweet] == False):
                tweets_text.append(data['text'][tweet])
                tweets_hashtags.append(get_hashtags(data))
            else:
                tweets_text.append(data['extended_tweet'][tweet]['full_text'])
                tweets_hashtags.append(get_hashtags(data['extended_tweet'][tweet]))
            
            tweets_date.append(data['created_at'][tweet])
            tweets_username.append(data['user'][tweet]['name'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
            tweets_likes.append(data['favorite_count'][tweet])
            tweets_retweets.append(data['retweet_count'][tweet])
        else:
            tweets_text.append(data['retweeted_status'][tweet]['text'])
            #We supposed it is the name and not the @name (screen_name)
            tweets_username.append(data['retweeted_status'][tweet]['user']['name']) 
            tweets_date.append(data['retweeted_status'][tweet]['created_at'])
            tweets_hashtags.append(get_hashtags((data['retweeted_status'][tweet])))
            tweets_likes.append(data['retweeted_status'][tweet]['favorite_count'])
            tweets_retweets.append(data['retweeted_status'][tweet]['retweet_count'])
            tweets_url.append("https://twitter.com/i/web/status/"+ data["id_str"][tweet])
        
    tweet_info = {'Tweet': tweets_text, 'Username': tweets_username, 'Date': tweets_date, 'Hashtags': tweets_hashtags, 
         'Likes': tweets_likes, 'Retweets': tweets_retweets, 'Url': tweets_url}

    return pd.DataFrame(data=tweet_info)

In [10]:
def decontracted(text: str) -> str:
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def punctuation_removal(text: str) -> str:
    tokenizer=RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(decontracted(text.lower()))

def stemming(text: list) -> list:
    ps = PorterStemmer()
    stemmed_text = []
    for word in text: 
        stemmed_text.append(ps.stem(word))
    return stemmed_text

def remove_stop_words(word_list: list) -> list:
    stop_words = set(stopwords.words('english'))
    stop_words.add('rt')
    text_without_stopwords = []
    for word in word_list:
        if not word in stop_words:
            text_without_stopwords.append(word)
    return text_without_stopwords

def text_normalization(text: str) -> str:
    text_normalized = punctuation_removal(text)
    text_stemmed  = stemming(text_normalized)
    text_without_stopwords = remove_stop_words(text_stemmed)
    return " ".join(text_without_stopwords)

In [11]:
#Test to get out rt:
#print(text_normalization('RT, Toca.Ovaris, 20.000 or 20,00'))

In [12]:
tweets=remove_unnecessary_columns(tweets)
tweets.head()

,created_at,id_str,text,source,truncated,in_reply_to_screen_name,user,retweeted_status,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,timestamp_ms,extended_tweet,extended_entities
0,Mon Nov 30 12:19:06 +0000 2020,1333385061723324419,"RT @briebriejoy: Are ""hunger and nutrition"" no...","<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 1204555554586128384, 'id_str': '1204555...",{'created_at': 'Mon Nov 30 04:58:09 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746468,NaN,NaN
1,Mon Nov 30 12:19:06 +0000 2020,1333385061719298049,"It is fair, and urgent. Esp. in KY, SC and Maine.","<a href=""http://twitter.com/download/iphone"" r...",False,None,"{'id': 627341141, 'id_str': '627341141', 'name...",NaN,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746467,NaN,NaN
2,Mon Nov 30 12:19:06 +0000 2020,1333385061970837504,RT @Rasmussen_Poll: Almost half (47%) of U.S. ...,"<a href=""http://twitter.com/download/android"" ...",False,None,"{'id': 3310664641, 'id_str': '3310664641', 'na...",{'created_at': 'Sun Nov 29 18:20:12 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746527,NaN,NaN
3,Mon Nov 30 12:19:06 +0000 2020,1333385062155513857,RT @Goss30Goss: When Trump is convicted &amp; ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,"{'id': 4081545435, 'id_str': '4081545435', 'na...",{'created_at': 'Mon Nov 30 03:18:51 +0000 2020...,0,0,0,"{'hashtags': [], 'urls': [], 'user_mentions': ...",False,False,low,1606738746571,NaN,NaN
4,Mon Nov 30 12:19:06 +0000 2020,1333385062193057799,"literally ""fall""\nhttps://t.co/uij0ZiPde0","<a href=""http://twitter.com/download/android"" ...",False,None,"{'id': 1331842561439199233, 'id_str': '1331842...",NaN,0,0,0,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",False,False,low,1606738746580,NaN,NaN


In [13]:
def inverted_index(data: pd.DataFrame)->dict:
    inv_index={}
    for doc_num in range(len(data['text'])):
        words=data['text'][doc_num].split()
        for word in words:
            if word in inv_index:
                if word not in inv_index[word]:
                    inv_index[word].append(doc_num)
            else:
                inv_index[word]=[doc_num]
    return inv_index

#REMOVE THIS FUNCTION, THERE MUST BE A LIBRARY THAT DOES THIS
def tf_idf_normalization(tf_idf: list) -> None:
    D_square=np.zeros(len(tf_idf))
    for i in range(len(tf_idf)):
        document_values=list(tf_idf[i].values())
        for j in range(len(document_values)):
            document_values[j]=(document_values[j]**2)
        D_square[i] = np.sum(document_values)
    
    for i in range (len(tf_idf)):
        for word in tf_idf[i]:
            tf_idf[i][word]= tf_idf[i][word]/np.sqrt(D_square[i])

def tf_idf(docs: list, inv_index: dict) -> list:
    tf=[]
    df={}
    idf={}
    tf_idf=[]
    N=len(tweets)
    for doc_text in docs:
        tf_doc_i={}
        words = doc_text.split()
        for word in words:
            if word in tf_doc_i:
                tf_doc_i[word]+=1/len(words)
            else:
                tf_doc_i[word]=1/len(words)
        tf.append(tf_doc_i)
        
    for word in inv_index:
        df[word]=len(inv_index[word])
        idf[word]=np.log(float(N/(df[word]+1)))

    for doc in range(len(docs)):
        tf_idf_per_doc={}
        for word in tf[doc]:
            tf_idf_per_doc[word]=tf[doc][word]*idf[word]
        tf_idf.append(tf_idf_per_doc)
        
    tf_idf_normalization(tf_idf)
    return tf_idf


In [14]:
def query_tfidf(query: str, query_vector: dict) -> None:
    query_tf_idf = tf_idf([query], inverted_index(tweets))
    for word in query.split():
        query_vector[word]= query_tf_idf[0][word]
    
def ranking_documents(query: str, tfidf: list):
    cosine_similarity={}
    i=0
    for document in tfidf:
        vectorized_doc = list(document.values())
        if all(word in document.keys() for word in query.split()):
            #THIS CAN BE IMPROVED
            vectorized_query = {key:0 for key in document.keys()}
            query_tfidf(query, vectorized_query)
            vectorized_query_values = list(vectorized_query.values())
            cosine_similarity[i] = np.dot(vectorized_query_values,vectorized_doc)
        else:
            cosine_similarity[i] = 0
        i+=1
    return cosine_similarity

In [15]:
database=get_tweet_data(tweets)
for i in range(len(tweets)):
    tweets['text'][i]=text_normalization(tweets['text'][i])

C:\Users\TESTER\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
database

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,"Are ""hunger and nutrition"" not issues in rural...",BJG 👑,Mon Nov 30 04:58:09 +0000 2020,None,2065,204,https://twitter.com/i/web/status/1333385061723...
1,"It is fair, and urgent. Esp. in KY, SC and Maine.",JHWBB,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385061719...
2,Almost half (47%) of U.S. likley voters now be...,Rasmussen Reports,Sun Nov 29 18:20:12 +0000 2020,None,16993,12098,https://twitter.com/i/web/status/1333385061970...
3,"When Trump is convicted &amp; imprisonedin NY,...",Andrew Goss 👊USAF👊,Mon Nov 30 03:18:51 +0000 2020,None,1781,202,https://twitter.com/i/web/status/1333385062155...
4,"literally ""fall""\nhttps://t.co/uij0ZiPde0",パンケーキ,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062193...
5,@_Mercian @realDonaldTrump Framed another way....,cinda collins,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062176...
6,If you're not already supporting @gaslitnation...,Mollie Katzen,Sat Nov 28 16:38:36 +0000 2020,None,159,38,https://twitter.com/i/web/status/1333385062029...
7,Today’s observations:\nBiden did not break his...,Warrior on Krak,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062440...
8,"So, we wake up to see that #StopTheStupid is t...",︻╦̵̵͇̿̿̿̿ Justice Johnson╤,Mon Nov 30 11:12:12 +0000 2020,None,66,22,https://twitter.com/i/web/status/1333385062583...
9,I will never give up on Trump.,LORI HENDRY,Mon Nov 30 00:50:59 +0000 2020,None,7200,1190,https://twitter.com/i/web/status/1333385062679...


In [17]:
tweets["text"].tolist()
tfidf=tf_idf(tweets["text"].tolist(), inverted_index(tweets))

In [87]:
print("Insert your query:\n")
query = input()
query=text_normalization(query)
docs = ranking_documents(query, tfidf)    
top = 20

database["score"]= pd.DataFrame(np.zeros(len(database)))
database["score"]=docs.values()
database_sorted=database.sort_values(by=["score"], ascending=False)
database_sorted.head(top)

#CUBRIR LOS CASOS DONDE NO SE LLEGA AL TOP 20 BUSCANDO MAS TWEETS
#Creo que ni la score ni gd_score tendrian que salir en el output (database), sinó en tweets database. 

Insert your query:

realDonaldTrump


,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,score,g(d),g(d)_normalized
1022,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,68557,14422,https://twitter.com/i/web/status/1333695821230...,0.494092,31130.333333,0.074230
1582,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,69307,14581,https://twitter.com/i/web/status/1333699462187...,0.494092,31468.000000,0.075036
1935,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,69291,14585,https://twitter.com/i/web/status/1333699539593...,0.494092,31464.666667,0.075028
1801,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,69313,14582,https://twitter.com/i/web/status/1333699509138...,0.494092,31470.500000,0.075042
1220,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,68565,14424,https://twitter.com/i/web/status/1333695862602...,0.494092,31134.000000,0.074239
1979,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,69291,14587,https://twitter.com/i/web/status/1333699548171...,0.494092,31465.666667,0.075030
1441,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,68570,14428,https://twitter.com/i/web/status/1333695912200...,0.494092,31137.833333,0.074248
1270,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,68566,14425,https://twitter.com/i/web/status/1333695874082...,0.494092,31134.833333,0.074241
1665,Only in America!,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,69279,14583,https://twitter.com/i/web/status/1333699480072...,0.494092,31459.666667,0.075016
1028,Only in America! https://t.co/s0sS1eYXJy,Donald J. Trump,Tue Dec 01 03:37:29 +0000 2020,None,68528,14424,https://twitter.com/i/web/status/1333695822471...,0.301420,31121.666667,0.074210


In [57]:
# get always full text by checking if truncated. If it is, then find extended_tweet or retweeted_status

In [81]:
def g_d_score(data: pd.DataFrame) -> list: #Hay que pasar tweets database
    #retweets -> 3/6; likes -> 2/6; replies -> 1/6
    #We did not consider the followers of the users, as it would biase (NO SE COMO SE ESCRIBE LO DEL BIASE) the importance
    #Of each tweet. Someone with a lot of followers could write something irrelevant to the query and unfairly, get a higher
    #punctuation than someone with not so many followers that wrote a relevant tweet (with a lot of likes/retweets/replies).
    gd_score=[]
    for row in data.iterrows():
        if str(row[1]['retweeted_status'])=='nan':
            retweets_score_score=row[1]['retweet_count']
            likes_score=row[1]['favorite_count']
            replies_score=row[1]['reply_count']
            score=((3/6)*int(retweets_score)+(2/6)*int(likes_score)+(1/6)*int(replies_score))
            gd_score.append(score)
        else:
            retweets_score=row[1]['retweeted_status']['retweet_count']
            likes_score=row[1]['retweeted_status']['favorite_count']
            replies_score=row[1]['retweeted_status']['reply_count']
            score=((3/6)*int(retweets_score)+(2/6)*int(likes_score)+(1/6)*int(replies_score))
            gd_score.append(score)
    return gd_score

In [86]:
gd_score=g_d_score(tweets)
database['g(d)']=gd_score
gd_score_norm = norm = [(float(i)-min(gd_score))/(max(gd_score)-min(gd_score)) for i in gd_score]
database['g(d)_normalized']=gd_score_norm
database

,Tweet,Username,Date,Hashtags,Likes,Retweets,Url,score,g(d),g(d)_normalized
0,"Are ""hunger and nutrition"" not issues in rural...",BJG 👑,Mon Nov 30 04:58:09 +0000 2020,None,2065,204,https://twitter.com/i/web/status/1333385061723...,0.000000,794.500000,0.001893
1,"It is fair, and urgent. Esp. in KY, SC and Maine.",JHWBB,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385061719...,0.000000,102.000000,0.000242
2,Almost half (47%) of U.S. likley voters now be...,Rasmussen Reports,Sun Nov 29 18:20:12 +0000 2020,None,16993,12098,https://twitter.com/i/web/status/1333385061970...,0.000000,11778.000000,0.028084
3,"When Trump is convicted &amp; imprisonedin NY,...",Andrew Goss 👊USAF👊,Mon Nov 30 03:18:51 +0000 2020,None,1781,202,https://twitter.com/i/web/status/1333385062155...,0.000000,708.000000,0.001687
4,"literally ""fall""\nhttps://t.co/uij0ZiPde0",パンケーキ,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062193...,0.000000,101.000000,0.000240
5,@_Mercian @realDonaldTrump Framed another way....,cinda collins,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062176...,0.000000,101.000000,0.000240
6,If you're not already supporting @gaslitnation...,Mollie Katzen,Sat Nov 28 16:38:36 +0000 2020,None,159,38,https://twitter.com/i/web/status/1333385062029...,0.000000,72.500000,0.000172
7,Today’s observations:\nBiden did not break his...,Warrior on Krak,Mon Nov 30 12:19:06 +0000 2020,None,0,0,https://twitter.com/i/web/status/1333385062440...,0.000000,19.000000,0.000044
8,"So, we wake up to see that #StopTheStupid is t...",︻╦̵̵͇̿̿̿̿ Justice Johnson╤,Mon Nov 30 11:12:12 +0000 2020,None,66,22,https://twitter.com/i/web/status/1333385062583...,0.000000,33.500000,0.000079
9,I will never give up on Trump.,LORI HENDRY,Mon Nov 30 00:50:59 +0000 2020,None,7200,1190,https://twitter.com/i/web/status/1333385062679...,0.000000,3024.000000,0.007210


In [21]:
os.remove(OUTPUT_FILENAME)

In [22]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN